In [52]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import math
from operator import add
import builtins
from google.cloud import storage
from collections import defaultdict

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [5]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [6]:
def open_gcp(file_name,dir_name):
    client = storage.Client(file_name)
    bucket = client.bucket(bucket_name)
    blob = bucket.get_blob(f'postings_gcp{dir_name}/' + file_name)
    return blob.open('rb')

def read_pickle(file_name,dir_name):
    stream = open_gcp(file_name+".pickle",dir_name)
    pick = pickle.load(stream)
    stream.close()
    return pick

def read_pkl(file_name,dir_name):
    stream = open_gcp(file_name+".pkl",dir_name)
    pick = pickle.load(stream)
    stream.close()
    return pick

In [7]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [8]:
bucket_name = '318457645' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)

In [9]:
porter = PorterStemmer()

In [10]:
from inverted_index_gcp import *

In [11]:
inverted_text = read_pkl('index_text','text')

In [12]:
inverted_title = read_pkl('index_title','title')

In [13]:
DL_dict = read_pickle('DL_title','title')
inverted_title.DL = DL_dict

In [14]:
# TUPLE_SIZE = 6       
# TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
# from contextlib import closing

# def read_posting_list(inverted, w, base_dir):
#     with closing(MultiFileReader()) as reader:
#         locs = inverted.posting_locs[w]
#         b = reader.read(locs, inverted.df[w] * TUPLE_SIZE, base_dir)
#         posting_list = []
#         for i in range(inverted.df[w]):
#             doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
#             tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
#             posting_list.append((doc_id, tf))
#         return posting_list

## Starting Search

In [54]:
def get_candidate_documents(query_to_search,index,base_dir):
    candidates = []    
    for term in np.unique(query_to_search):
        if term in index.df.keys():        
            current_list = index.read_posting_list(term,base_dir )               
            candidates += current_list    
    candidates = set([x[0] for x in candidates])
    return set(candidates)

In [55]:
import math
from itertools import chain
import time
# When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.
class BM25_from_index:
    """
    Best Match 25.    
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self,index,base_dir='',k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.N = len(index.DL)
        self.AVGDL = builtins.sum(index.DL.values())/self.N
        self.base_dir = base_dir

    def calc_idf(self,list_of_tokens):
        idf = {}        
        for term in list_of_tokens:            
            if term in self.index.df.keys():
                n_ti = self.index.df[term]
                idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
            else:
                pass                             
        return idf
        

    def search(self, queries,N=100):
        dict_B25 = {}
        unique_terms_in_queries = []
        unique_terms_in_queries = [q for q in queries.values()]
        unique_terms_in_queries = [item for sublist in unique_terms_in_queries for item in sublist]
        unique_terms_in_queries = [porter.stem(x) for x in unique_terms_in_queries]
        unique_terms_in_queries = list(set(unique_terms_in_queries))
        idf = self.calc_idf(unique_terms_in_queries)
        self.idf = idf
        pls_dict = {}
        for q_id,q in queries.items():
            q = [porter.stem(x) for x in q]
            for term in q:
                if term in self.index.df.keys() and term not in pls_dict:  
                    pls_dict[term] = dict(self.index.read_posting_list(term ,self.base_dir))
            candidates = get_candidate_documents(q, self.index,self.base_dir)
            dict_B25[q_id] = [(c, self._score(q,c,pls_dict)) for c in candidates]
            dict_B25[q_id] = sorted(dict_B25[q_id] , key = lambda x: x[1], reverse = True)[:N]
        return dict_B25

    def _score(self,query, doc_id, pls_dict):  
        score = 0.0        
        doc_len = self.index.DL[doc_id]        
        for term in query:
            if term in self.index.df.keys():                 
                if doc_id in pls_dict[term].keys():            
                    freq = pls_dict[term][doc_id]
                    numerator = self.idf[term] * freq * (self.k1 + 1)
                    denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                    score += (numerator / denominator)
        return score

In [56]:
bm25_text = BM25_from_index(inverted_text,'text')

In [57]:
dict_query = {}
dict_query[1] = ['how','to', 'vote']
dict_query[2] = ['python']
dict_query[3] = ['dim', 'sum']
dict_query[4] = ['michelin']
dict_query[5] = ['what', 'is', 'critical', 'race' ,'theory']
dict_query[6] = ['what', 'to', 'watch']
dict_query[7] = ['best', 'marvel', 'movie']
dict_query[8] = ['yoga']
dict_query[9] = ['nba']
dict_query[10] = ['chocolate']
dict_query[11] = ['how' ,'tall', 'is' ,'the' ,'eiffel' ,'tower']
dict_query[12] = ['best', 'ice', 'cream', 'flavour']
dict_query[13] = ['how' ,'to' ,'tie' ,'a' ,'tie']
dict_query[14] = ['how' ,'to' ,'earn' ,'money' ,'online']
dict_query[15] = ['google', 'trends']

In [58]:
def merge_results(title_scores,body_scores,title_weight=0.5,text_weight=0.5,N = 100):    
        dict_topN = {}
        for q_id in title_scores:
            tuples_title = title_scores[q_id]
            doc_id_title = [x[0] for x in tuples_title]

            tuples_body = body_scores[q_id]
            doc_id_body = [x[0] for x in tuples_body]

            list_diff_id = [x for x in doc_id_title if x not in doc_id_body] + [x for x in doc_id_body if x not in doc_id_title]
            list_same_id = [x for x in doc_id_title if x not in list_diff_id]

            tuples_diff = [(x[0], x[1] * title_weight) for x in tuples_title if x[0] in list_diff_id] + [(x[0], x[1] * text_weight) for x in tuples_body if x[0] in list_diff_id]

            dict_same = {}
            for x, y in tuples_title: 
                if(x in list_same_id):
                    dict_same.setdefault(x, []).append(y)
            for x, y in tuples_body: 
                if(x in list_same_id):
                      dict_same.setdefault(x, []).append(y)
            tuples_same = [(x, y[0] * title_weight + y[1] * text_weight) for x, y in dict_same.items()]

            new_tuples = tuples_same + tuples_diff
            new_tuples = sorted(new_tuples, key=lambda t: t[1], reverse=True)[:N]
            dict_topN[q_id] = new_tuples
        return dict_topN

In [59]:
bm25_title = BM25_from_index(inverted_title,'title')


In [60]:
lst_vote=[17255946, 99860, 66293350, 29066482, 924170, 85533, 994892, 1127534, 430980, 21491579, 667785, 382165, 1591777, 64210828, 479210, 1262999, 319703, 7209753, 12119521, 1857, 37179209, 70322, 630522, 30332, 41609161, 1236844, 22265206, 20071428, 318313, 7928434, 538393, 27363348, 907259, 699156, 33110953, 1833715, 79906, 6102876, 886827, 1125781, 55791, 20089569, 823320, 9457, 863306, 25767045, 29589, 70136, 23298, 925493, 5048642, 14650035, 59594788, 26726864, 925506, 480905, 1387946, 925740, 20072937, 1050340, 7959, 23269223, 64499620, 925728, 371301, 763947, 31664, 924141, 46023, 5372935, 57393329, 48870396, 315204, 23032578, 25908, 37593868, 871796, 64695698, 51582, 36968102, 8009590, 1516039, 7529001, 169131, 1435428, 24909346, 47350955, 13624574, 1141303, 33849881, 827717, 1442241, 7913597, 261709, 65028817, 60670913, 52647644, 1435365, 1125761]

In [61]:
lst_python = [23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462]

In [62]:
lst_dim_sum= [100640, 269558, 1959777, 22670461, 67072363, 11971218, 34617580, 47775306, 47840375, 28827117, 11827767, 17031486, 11689293, 9526854, 11980851, 11971397, 47806602, 11266183, 4992439, 47775348, 46357913, 52906, 3768042, 3746367, 47827570, 41905327, 1844187, 4023588, 2474625, 4749106, 63602774, 4146044, 3828139, 46186668, 47827574, 37967188, 10265984, 5321303, 46665704, 4344526, 11894026, 1907296, 60769053, 519667, 18408298, 13958538, 13719853, 41546279, 67493391, 3577886, 2054954, 48241318, 4093674, 898916, 2012983, 13902799, 2626421, 54284514, 10887219, 40759810, 20505468, 43607423, 6168739, 2134361, 47434601, 47769544, 19433498, 47837049, 52554299, 678353]

In [63]:
lst_michelin = [2036409, 79732, 35052231, 644781, 50991931, 51729995, 56721897, 60583278, 4512778, 53866975, 31728660, 636344, 48286897, 2550824, 66591573, 1360573, 52882803, 1761526, 56758995, 52971602, 2575380, 59519477, 1343949, 53748793, 66029904, 17284321, 62380392, 8250222, 1044117, 37032671, 51478870, 11216001, 2947322, 60085976, 1291991, 9483388, 11231759, 22513329, 5046302, 35927164, 43610835, 2074655, 52076814, 66846904, 58067594, 3997367, 9547083, 16348889, 16184595, 44972706, 37026554, 36753304, 65601792, 66605355, 34964813, 10873990, 13235623, 46302846, 37297021, 3463398, 32739936, 2337323]

In [64]:
lst_critical = [2002497, 1655005, 66355340, 650704, 42736966, 4730170, 41055774, 56081344, 2570959, 3322361, 818420, 36598455, 14684661, 9557957, 37989952, 46269496, 1943640, 46677020, 10782332, 52985524, 475952, 65366095, 13700748, 742316, 3923466, 4080953, 65176113, 8848402, 487090, 56833348, 64159270, 25613, 48253536, 14062336, 55536490, 53653129, 63327652, 43441237, 141013, 2612612, 4876011, 35237477, 64444618, 3057990, 50777038, 67071821, 56042524, 295993, 10827381, 356785, 1849681, 200734, 3188121, 25614, 2495537, 67924707, 42347365, 26952693, 6013, 42757076, 33637422, 60698570, 12210793, 40845811, 46613990, 56363941, 64784613, 22494577, 2794874, 1566357, 229236, 23213, 33178, 348111, 67162622, 40068424, 8094067, 1896889, 1999636, 1929338, 42469790]

In [65]:
lst_watch = [55840499, 40475635, 48308444, 65766671, 175537, 7235116, 164376, 42262371, 60883, 53066659, 989858, 302808, 37331423, 18948312, 4552880, 13612929, 55864774, 35729649, 1350109, 52766120, 147143, 55411770, 6920292, 38539, 29831, 23776, 20840, 50276487, 3470969, 59480957, 617668, 33279717, 63704575, 65693607, 57944031, 27306717, 1732642, 32255505, 37061023, 43379979, 19508643, 58150649, 2381862, 21899417, 40438099, 40030145, 6405787, 504230, 925736, 60446377, 37669357, 44146329, 54647064, 66938917, 64704775, 169672, 40867822, 5230872, 66259016, 67736156, 13690215, 342398, 57579192, 394697, 33757091, 55825336, 1068736, 2009711, 47778072, 41451023, 35306910, 31607, 56789255, 18383702, 1492414, 37901793, 2650227, 7152087, 60587209, 818234, 66814247, 59800766, 2387993, 66255918, 63241742, 4765530, 27375211, 21863425, 618856]

In [66]:
lst_marvel = [57069491, 65967176, 42163310, 878659, 27306717, 41677925, 1074657, 44240443, 17296107, 60952488, 43603241, 22114132, 46208997, 36450985, 41974555, 56289672, 60616450, 59502488, 33038861, 61699239, 61651800, 39368416, 29129051, 55935213, 54537218, 62372638, 60774345, 63090183, 37497391, 51430647, 67229718, 9110929, 61329320, 44254295, 41974496, 200563, 58481694, 48530084, 56289572, 22144990, 612052, 59162931, 55511148, 55511147, 61073786, 59892, 36484005, 36484254, 66423851, 62482816, 1275470, 5676692, 7927053, 60754840, 26999426, 60744481, 56289553, 60463979, 701741, 60283633, 1129847, 36439749, 4451883, 55511155, 22144721, 45359871, 723126, 43655965, 57275457, 12673434, 43867095, 26763420, 39293265, 15003874, 41668588, 61592102, 67063919, 11891433, 64057670, 61513780, 39345917, 67063906, 1221476, 41008758, 60587000, 7729, 2152196, 5027882, 509738, 403585, 26866372, 1339248, 3473503, 4148655]

In [67]:
lst_yoga = [34258, 59580357, 315141, 47035547, 626718, 32016617, 59516988, 61583621, 38424, 196789, 734272, 60851303, 988753, 92222, 419015, 60992198, 621809, 83742, 3127300, 23834216, 1381751, 744158, 18911094, 1817874, 59052395, 43562594, 31793047, 6154795, 17452177, 62342792, 13936750, 60805249, 60823333, 59899027, 46719542, 7702313, 60310807, 44270128, 68489731, 60500106, 1652601, 359983, 60300514, 45482242, 34756533, 58520592, 43611227, 47586513, 1775374, 6933106, 13412771, 62549131, 49278248, 62557135, 4523354, 60468426, 666420, 1017009, 35795560, 34843274, 60199413, 16488467, 29828807, 585681, 41757168, 66018807, 61285582, 1226448, 28848113, 11487904, 4242777, 1661867, 10671559, 44035836, 6931929, 60705564, 61690747, 7343803, 9042644, 36991237, 3965223, 63090179, 632990, 1041815, 53087254, 418999, 1632919]

In [68]:
lst_nba = [22093, 16795291, 65166616, 65785063, 835946, 890793, 3921, 450389, 20455, 987153, 240940, 246185, 9000355, 5608488, 3280233, 3505049, 5958023, 72852, 8806795, 1811390, 2423824, 516570, 15392541, 72893, 412214, 278018, 12106552, 42846434, 12754503, 9807715, 4108839, 33328593, 64063961, 7215125, 1811320, 1111137, 5035602, 60483582, 9397801, 255645, 16899, 43376, 72855, 65785040, 72866, 6215230, 4987149, 72878, 16160954, 243389, 64639133, 38958735, 72858, 27196905, 38153033, 1385825, 9733533, 49926096, 4875689, 4750398, 28754077, 43569250, 22092, 72889, 59798759, 49778089, 346029, 8588996, 1956255, 52454088, 25390847, 31667631, 878666, 48695845, 72857, 459304, 27837030, 17107550, 72861, 54859596, 9195892, 6560301, 72875, 72883, 240989, 3196517, 24612090]

In [69]:
lst_chocolate = [7089, 6693851, 6672660, 23159504, 49770662, 167891, 2399742, 100710, 76723, 5290678, 54229, 3881415, 3720007, 32652613, 1471058, 5239060, 1421165, 1730071, 1277918, 7419133, 17720623, 1765026, 19079925, 1979873, 497794, 57947, 15007729, 85655, 4250574, 2386481, 228541, 55225594, 318873, 22631033, 27767967, 7061714, 8175846, 3881265, 3046256, 606737, 845137, 16161419, 3098266, 54573, 11323402, 936243, 39280615, 13687674, 47155656, 7151675, 43627595, 26879832, 43098662, 2333593, 349448, 2052211, 4432842, 56412300, 1411016, 2152015, 3502051, 33372192, 61422444, 2385217, 1217747, 24315397, 7082459, 856246, 6050655, 27162455, 52140446, 37243595, 36961531, 245067, 1148978, 1770825, 976322, 10300434, 7249348, 14945749, 62851606, 637004, 16224368, 18509922]

In [70]:
lst_eifel = [9232, 31655286, 4465969, 50230910, 35392555, 17664319, 548182, 148869, 34080, 64487, 12232, 38253534, 1278990, 705310, 1849140, 37882040, 305175, 41731654, 5486819, 38345627, 222491, 48534532, 61371, 5065635, 1273047, 881491, 40797848, 6788, 10000, 595149, 43617005, 64437720, 580320, 24001, 5052869, 5028853, 6112, 43825684, 22910214, 13338662, 16990777, 39902695, 7424280, 2040288, 3945278, 167585, 1632912, 13065408, 59917348, 27218900, 1884312, 4539416, 71899, 38342244, 308868, 12629761, 61402898, 19931851, 66273148, 384286, 5763069, 19437928, 8166922, 4649105, 1525546, 26775738, 671331, 2730731]

In [71]:
lst_icecream = [42416319, 30224834, 48212, 21828, 745000, 3342275, 23054130, 25311310, 2837891, 1124192, 978236, 1188562, 7082459, 47270314, 3651805, 2606761, 40783886, 34482422, 2298022, 6311077, 43334967, 24056098, 483473, 2190209, 11859370, 44309134, 17634819, 54595558, 39817115, 1889217, 897934, 2142071, 36906301, 1412877, 3644514, 42360671, 53354661, 22757645, 23114830, 2831887, 548336, 566114, 872876, 12209015, 42393261, 515436, 657055, 19558531, 8703840, 58534087, 50727159, 1532857, 362124, 40790752, 1092688, 5266044, 1531621, 37652572, 1207268, 1493164, 14342785, 7660130, 64594706, 2446518, 16969340, 20439501, 29538871, 18944151, 60888779, 33481262, 50498250, 430008, 2246339, 60395596, 6889129]

In [72]:
lst_tie = [6974438, 298232, 1517154, 6159650, 491172, 2881613, 6719413, 38464202, 2881547, 609528, 435492, 30597745, 37469, 59024, 371327, 2078529, 8491563, 40536338, 37421, 40774543, 5118574, 2766942, 4673201, 2636347, 559339, 20327832, 17412253, 426315, 6060889, 2110221, 17566205, 529780, 337814, 5270659, 7273911, 249858, 32468394, 16887435, 903052, 4676317, 47419336, 49355, 1807656, 19960341, 44547, 7775525, 48253954, 29604056, 469664, 9066654, 8133498, 52281147, 24997576, 37454, 9952602, 16938429, 7154355, 22199452, 912227, 22549390, 1632324, 9675171, 8867983, 3294758, 57591892, 5563310, 2297090, 7718500, 35973111, 50795, 4553108, 4660251, 20323494, 3493102, 5302000, 3013373, 14438844, 498076, 8100167, 12356944, 4169317, 3664299, 801978, 42980570, 27833186]

In [73]:
lst_money = [17362858, 8957449, 13612149, 51895777, 60898024, 50683920, 42471422, 8983183, 63295665, 484030, 3206057, 1847385, 52412294, 31452088, 1012806, 59440349, 195809, 703758, 15037, 1279101, 2095532, 17418777, 805626, 6050003, 19390, 19776474, 11921, 4193788, 14754518, 21179478, 2732267, 2941963, 39217006, 2905253, 1946204, 3287760, 59872594, 39206, 63121, 234921, 48732, 19344654, 372478, 32843712, 4361873, 806538, 3524766, 2797868, 43601944, 48449792, 13051060, 45691897, 46526, 36662188, 142528, 19288592, 43250171, 42332846, 2213921, 56440839, 33680356, 529056, 32362461, 68186473, 51097615, 17519943, 4567010, 610452, 60739751, 19585449, 3528947, 846772, 59568301, 10554676, 2738057, 28026533, 26700112, 1549819, 35917094, 24593449, 30810141, 4992612, 57851653, 32400932, 18169326, 21549213, 57275912, 1118379, 57313961, 909036, 35356507, 4355398, 236424, 1719292, 52328, 1184796]

In [74]:
lst_google = [5101133, 18745215, 9483388, 24615479, 12431, 37593992, 10448121, 6956596, 1497849, 286747, 59868408, 4465875, 8598331, 35673556, 27051151, 1360154, 47799755, 18968719, 44008170, 24213566, 54737654, 5339378, 187946, 26275110, 30853394, 12003767, 7043619, 67414554, 47615485, 32613098, 33367993, 34083096, 3609466, 49132107, 53356243, 28236187, 1027207, 47339470, 37779362, 37980517, 62705077, 41815118, 3524766, 37728339, 32158961, 1491100, 56313056, 3660182, 1762557, 9933471, 43959031, 38755390, 3067569, 28190146, 37670240]

In [124]:
dict_results = {}
dict_results[1] = lst_vote
dict_results[2] = lst_python
dict_results[3] = lst_dim_sum
dict_results[4] = lst_michelin
dict_results[5] = lst_critical
dict_results[6] = lst_watch
dict_results[7] = lst_marvel
dict_results[8] = lst_yoga
dict_results[9] = lst_nba
dict_results[10] = lst_chocolate
dict_results[11] = lst_eifel
dict_results[12] = lst_icecream
dict_results[13] = lst_tie
dict_results[14] = lst_money
dict_results[15] = lst_google


In [125]:
bm25_queries_score_train_title = bm25_title.search(dict_query,100)
bm25_queries_score_train_body = bm25_text.search(dict_query,100)

In [126]:
def avg_pr_query(w1,w2):
    search_result_dic = merge_results(bm25_queries_score_train_title,bm25_queries_score_train_body,w1,w2)
    dict_eq = {}
    dict_avg_p = {}
    for k,v in search_result_dic.items():
        lst = [x[0] for x in v]
        dict_eq[k] = len([x for x in lst if x in dict_results[k]])
        dict_avg_p[k] = average_precision(dict_results[k], lst)
    total = builtins.sum([v for k,v in dict_avg_p.items()])/15
    return total

In [127]:
dict_map = {}
dict_map[(0.5,0.5)] = avg_pr_query(0.5,0.5)
dict_map[(0.63,0.37)] = avg_pr_query(0.63,0.37)
dict_map[(0.60,0.4)] = avg_pr_query(0.60,0.4)
dict_map[(0.2,0.8)] = avg_pr_query(0.2,0.8)
dict_map[(0.4,0.6)] = avg_pr_query(0.4,0.6)
dict_map[(0.3,0.7)] = avg_pr_query(0.3,0.7)
dict_map[(0.3,0.7)] = avg_pr_query(0.3,0.7)

In [128]:
print(dict_map)

{(0.5, 0.5): 0.49726666666666663, (0.63, 0.37): 0.45820000000000005, (0.6, 0.4): 0.47246666666666665, (0.2, 0.8): 0.49500000000000005, (0.4, 0.6): 0.5284, (0.3, 0.7): 0.5149333333333334}


In [122]:
def precision_at_k(true_list,predicted_list,k=40):        
    sum1 = 0
    if k > len(predicted_list):
        k = len(predicted_list)
    for i in range(k):
        if predicted_list[i] in true_list:
            sum1 +=1
    return builtins.round(sum1/k,3)

In [121]:
def average_precision(true_list,predicted_list,k=40):
    sum1 = 0    
    if k > len(predicted_list):
        k = len(predicted_list) 
    count = 0 
    for i in range(k):
        if (predicted_list[i] in true_list):
            count+=1
            sum1+= precision_at_k(true_list,predicted_list,i+1)
    if count == 0:
        return builtins.round(0, 3)
    return builtins.round(sum1/count, 3)

In [ ]:

lst_equal = [x for x in search_result_lst if x in lst_michelin]
print(len(lst_equal))

In [ ]:
lst_indecies = [search_result_lst.index(x) for x in lst_michelin if x in search_result_lst]
print(lst_indecies)

In [ ]:
print(porter.stem('michelin'))
print(sorted(inverted_title.read_posting_list('vote', 'title'),key=lambda x : x[1], reverse=True)[:100])